In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
c_data = pd.read_csv("cities_weather.csv")
c_data

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Tuktoyaktuk,CA,1600563790,69.45,-133.04,48.20,76,75,12.75
1,Carnarvon,AU,1600563928,-24.87,113.63,62.60,55,90,14.99
2,Caravelas,BR,1600563929,-17.71,-39.25,73.36,82,0,9.01
3,Gushikawa,JP,1600563929,26.35,127.87,82.40,83,20,5.82
4,Atherton,GB,1600563929,53.52,-2.49,54.00,80,14,1.99
...,...,...,...,...,...,...,...,...,...
706,Bedi,IN,1600565000,22.50,70.05,82.08,81,42,8.19
707,Dalby,AU,1600565000,-27.18,151.27,75.20,50,40,12.75
708,Teupasenti,HN,1600565000,14.22,-86.70,77.00,61,40,2.24
709,Christchurch,NZ,1600564743,-43.53,172.63,57.99,67,0,14.99


In [3]:
data = c_data.dropna()
data

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Tuktoyaktuk,CA,1600563790,69.45,-133.04,48.20,76,75,12.75
1,Carnarvon,AU,1600563928,-24.87,113.63,62.60,55,90,14.99
2,Caravelas,BR,1600563929,-17.71,-39.25,73.36,82,0,9.01
3,Gushikawa,JP,1600563929,26.35,127.87,82.40,83,20,5.82
4,Atherton,GB,1600563929,53.52,-2.49,54.00,80,14,1.99
...,...,...,...,...,...,...,...,...,...
706,Bedi,IN,1600565000,22.50,70.05,82.08,81,42,8.19
707,Dalby,AU,1600565000,-27.18,151.27,75.20,50,40,12.75
708,Teupasenti,HN,1600565000,14.22,-86.70,77.00,61,40,2.24
709,Christchurch,NZ,1600564743,-43.53,172.63,57.99,67,0,14.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = data[["Lat","Lng"]]

#Store humidity values in humidity
humidity = data["Humidity"]
maxhum = humidity.max()

In [5]:
# Plot Heatmap
heatmap = gmaps.figure()

layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhum, point_radius=3)

heatmap.add_layer(layer)

# Display
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

### Ideal Weather Conditions
* Temperature: [80-100]
* Cloudiness: [10-20]
* Humidity: [60-85]

In [6]:
# Narrow Data Based on Ideal Weather Conditions
ideal = data.loc[(data["Max Temp"] > 80) & (data["Max Temp"] < 100) & (data["Cloudiness"] > 10) & (data["Cloudiness"] < 20) & (data["Humidity"] > 60) & (data["Humidity"] < 85)]
ideal

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
186,Butaritari,KI,1600564208,3.07,172.79,82.26,73,11,10.56
196,Kerema,PG,1600564210,-7.96,145.78,83.10,72,16,6.02
252,Dingle,PH,1600564115,11.00,122.67,84.94,77,12,6.11
273,Pedernales,DO,1600564310,18.04,-71.74,82.20,71,12,2.26
441,Pandan,PH,1600564584,14.05,124.17,83.88,81,18,8.84
509,Basco,PH,1600564659,20.45,121.97,83.71,77,14,11.97
540,Kalianget,ID,1600564725,-7.05,113.93,84.11,71,11,13.82
647,Marsh Harbour,BS,1600564928,26.54,-77.06,84.15,74,17,5.08


### Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Save Ideal Cities
hotel_df = pd.DataFrame(ideal)
hotel_df.reset_index(drop=True, inplace=True)

In [17]:
# Set Parameters
params = {"type": "lodging", "radius": 5000, "key": g_key}

# Use Coordinates for Locations
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # Add Location into params
    params["location"] = f"{lat},{lng}"
    
    # Base URL
    base_url =  "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Request API Calls
    hotel = requests.get(base_url, params=params)

    # Convert to JSON
    hotel = hotel.json()
    
    # Find Hotel Closest to Coordinates
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Hotel unavailable. Skipping...")
            
hotel_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Butaritari,KI,1600564208,3.07,172.79,82.26,73,11,10.56,Isles Sunset Lodge
1,Kerema,PG,1600564210,-7.96,145.78,83.10,72,16,6.02,Elore Lodge
2,Dingle,PH,1600564115,11.00,122.67,84.94,77,12,6.11,Camp Pasica
3,Pedernales,DO,1600564310,18.04,-71.74,82.20,71,12,2.26,Hotel Costa Marina
4,Pandan,PH,1600564584,14.05,124.17,83.88,81,18,8.84,Bonifacio Lodge
5,Basco,PH,1600564659,20.45,121.97,83.71,77,14,11.97,Nathaniel's Lodge
6,Kalianget,ID,1600564725,-7.05,113.93,84.11,71,11,13.82,Pesanggrahan Garam
7,Marsh Harbour,BS,1600564928,26.54,-77.06,84.15,74,17,5.08,Abaco Beach Resort


In [20]:
# Format Markers Layers
locs = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locs)


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
h_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locs = hotel_df[["Lat", "Lng"]]

In [25]:
# Format Hotel Location Layer
hotels_layer = gmaps.symbol_layer(locs, fill_color="red", stroke_color="black", scale=5, info_box_content=h_info)

In [27]:
# Add Layers to Map
heatmap.add_layer(markers)
heatmap.add_layer(hotels_layer)
heatmap

Figure(layout=FigureLayout(height='420px'))